### Kalman Filter applied in market beta

In [4]:
### Importing librarys
import pandas as pd
import numpy as np
from pykalman import KalmanFilter
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid') #### Plot style

In [6]:
### Importing data from economatica
df = pd.read_excel('C:/Users/Jhona/OneDrive - Grupo Marista/TCC/Dados (Temporário)/retornos_diarios.xlsx', skiprows=3, 
                    parse_dates=True, index_col=0)

In [7]:
df.head()

,Fechamento\najust p/ prov\nEm moeda orig\nABCB4,Fechamento\najust p/ prov\nEm moeda orig\nALPA4,Fechamento\najust p/ prov\nEm moeda orig\nABEV3,Fechamento\najust p/ prov\nEm moeda orig\nAMER3,Fechamento\najust p/ prov\nEm moeda orig\nAURE3,Fechamento\najust p/ prov\nEm moeda orig\nB3SA3,Fechamento\najust p/ prov\nEm moeda orig\nBPAN4,Fechamento\najust p/ prov\nEm moeda orig\nBEES3,Fechamento\najust p/ prov\nEm moeda orig\nBRSR6,Fechamento\najust p/ prov\nEm moeda orig\nBRML3,...,Fechamento\najust p/ prov\nEm moeda orig\nUSIM5,Fechamento\najust p/ prov\nEm moeda orig\nVALE3,Fechamento\najust p/ prov\nEm moeda orig\nVLID3,Fechamento\najust p/ prov\nEm moeda orig\nVIIA3,Fechamento\najust p/ prov\nEm moeda orig\nVIVR3,Fechamento\najust p/ prov\nEm moeda orig\nVULC3,Fechamento\najust p/ prov\nEm moeda orig\nWEGE3,Fechamento\najust p/ prov\nEm moeda orig\nWHRL4,Fechamento\najust p/ prov\nEm moeda orig\nPORT3,Fechamento\najust p/ prov\nEm moeda orig\nYDUQ3
Data,,,,,,,,,,,,,,,,,,,,,
2010-12-31,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
2011-01-03,6.657781,4.752996,5.997015,27.145312,-,2.744426,3.51241,1.414426,7.993412,9.397272,...,15.720192,28.41102,11.204076,-,2170.290674,26.450479,2.553589,1.467972,3.280319,6.404468
2011-01-04,6.671213,4.766236,5.998394,27.320228,-,2.750571,3.555349,1.41208,7.890703,9.262625,...,16.343099,28.999487,11.342398,-,2151.30854,25.978149,2.509441,1.464068,3.296223,6.392475
2011-01-05,6.733895,4.810368,5.886725,26.903761,-,2.68708,3.520998,1.388624,8.01574,9.313117,...,16.720121,29.523124,11.342398,4.4721,2138.653784,27.376246,2.47575,1.460164,3.254474,6.416462
2011-01-06,6.604053,4.814781,5.83158,26.978725,-,2.728042,3.417944,1.388624,7.886238,9.257014,...,16.597179,29.144111,11.480719,4.4721,2138.653784,27.206207,2.449029,1.460164,3.275348,6.378083
